In [ ]:
from brian2 import *
import numpy as np
import logging, warnings

prefs.codegen.target = 'cython'
set_device('runtime')
warnings.filterwarnings('ignore', category=RuntimeWarning)  # check error later
np.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)



start_scope()

defaultclock.dt = 0.001*ms  

# @implementation('numpy', discard_units=True)
# @check_units(value=1, result=1)
# def shifter(value):
#     """
#     Custom test function to compute a value based on weight, time, and spikes received.
#     This is a placeholder for the actual computation logic.
#     """
#     #print("value= ", value)
#     return value + 0.004

# Custom timing function
@implementation('numpy', discard_units=True)
@check_units(w=1, time=1, spike_received=1, result=1)
def spike_timing(w, time):
    #print("spikes recived" , spikes_received)
    #print("weight= ", w)
    x = (time % 1)
    #print("x= ", x)
    z = 5.0 * (x - 0.5)  # Smoother sigmoid
    #print("z= ", z)
    sigmoid_val = 1.0 / (1.0 + np.exp(-w * z))
    # if sigmoid_val < layer:
    #     return layer
    return sigmoid_val

@implementation('numpy', discard_units=True)
@check_units(w=1, time=1, result=1)
def d_spike_timing_dw(w, time):
    x = (time % 1)
    z = 5.0 * (x - 0.5)
    sig = 1.0 / (1.0 + np.exp(-w * z))
    return sig * (1.0 - sig) * z



def mini1x1x1(inputs, w1, w2):
    """
    Mini 1x1x1 network with a single input, hidden, and output neuron.
    This is a minimal example to demonstrate the basic structure of a spiking neural network.
    """
    # Define network sizes
    n_input = 1
    n_hidden = 1
    n_output = 1
    n_total = n_input + n_hidden + n_output

    # Full neuron group
    neurons = NeuronGroup(n_total, '''
        v : 1
        sum : 1
        spikes_received : 1
        scheduled_time : second
        global_clock : 1
        spiked : boolean
    ''', threshold='v > 1', reset='''
    v = 0
    spiked = True
    ''', method='exact')

    neurons.v = 0
    neurons.scheduled_time = 1e9 * second
    neurons.global_clock = 0.0
    neurons.sum = 0.0
    neurons.spikes_received = 0.0

    # Spike inputs (one per input neuron)     
    stim = SpikeGeneratorGroup(n_input, indices=range(n_input), times=(inputs) * ms)

    # Input → Hidden connections
    syn_input = Synapses(stim, neurons[0:n_input], '''
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += t/ms
        scheduled_time = ((sum/spikes_received) + layer) * ms 
    ''')
    syn_input.connect(j='i')  # connect stim[i] to neurons[i]
    syn_input.layer = 0
    #syn_input.w = -42  # just to skip and return t in function
    

    # Hidden layer: input → hidden
    syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input + n_hidden], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock)
        scheduled_time = ((sum/spikes_received) + layer + 0.004) * ms 
    ''')
    syn_hidden.connect()
    syn_hidden.w = w1 
    syn_hidden.layer = 1

    # Output layer: hidden → output
    syn_output = Synapses(neurons[n_input:n_input + n_hidden], neurons[n_input + n_hidden:n_total], '''
        w : 1
        layer : 1
    ''', on_pre='''
        spikes_received += 1
        sum += spike_timing(w, global_clock)
        scheduled_time = shifter((sum/spikes_received) + layer + 0.004) * ms
        
    
    ''')
    syn_output.connect()
    syn_output.w = w2 
    syn_output.layer = 2

    neurons.run_regularly('''
        v = (1.0 - spiked) * int(abs(t - scheduled_time) < 0.005*ms) * 1.2
        global_clock += 0.001
        
    ''', dt=0.001*ms)

        #v = int(abs(t - scheduled_time) < 0.005*ms) * 1.2

    # Monitors
    mon = StateMonitor(neurons, 'v', record=True, dt=0.01*ms)
    mon_sum = StateMonitor(neurons, 'sum', record=True)
    sp_mon = StateMonitor(neurons, 'spikes_received', record=True)
    sch_time = StateMonitor(neurons, 'scheduled_time', record=True)


    spikemon = SpikeMonitor(neurons)


    run(5*ms)

    # Plot voltages
    figure(figsize=(10, 6))
    for i in range(n_total):  # All neurons
        plot(mon.t/ms, mon.v[i], label=f'Neuron {i}')
    xlabel('Time (ms)')
    ylabel('Membrane potential')
    legend()
    title('SNN Spike Propagation Across All Layers')
    show()

    plot(mon_sum.t/ms, mon_sum.sum[3])  # or any neuron index
    print(mon_sum.sum[1])
    print(sp_mon.spikes_received[1])
    print(sch_time.scheduled_time[1])

    for i in range(n_total):
        times = spikemon.spike_trains()[i]
        if len(times) > 0:
            formatted_times = [f"{t/ms:.3f} ms" for t in times]
            print(f"Neuron {i} spike times: {formatted_times}")


    # To get the postsynaptic neuron, you would need to access the Synapses object
    # For example, if you have a Synapses object called 'syn', you can do:
    # postsynaptic_neurons = syn.i[syn.j == i]
    # print(f"Spike from Neuron {i} is traveling to Neurons: {postsynaptic_neurons}")

            
    return {
    'spike_trains': spikemon.spike_trains(),
    }



print(mini1x1x1([0.8], 3.0, -0.5))

Neuron 0 spike times: ['0.801 ms']
Neuron 1 spike times: ['1.989 ms']
Neuron 2 spike times: ['2.231 ms']
{'spike_trains': {0: array([0.801]) * msecond, 1: array([1.989]) * msecond, 2: array([2.231]) * msecond}}


: 

In [56]:
mini1x1x1([0.5], 0.5, 0.5)

Neuron 0 spike times: ['0.501 ms']
Neuron 1 spike times: ['1.497 ms']
Neuron 2 spike times: ['2.494 ms']


{'spike_trains': {0: array([0.501]) * msecond,
  1: array([1.497]) * msecond,
  2: array([2.494]) * msecond},
 'scheduled_times': array([[1.00000e+06, 1.00000e+06, 1.00000e+06, ..., 5.00000e-07,
         5.00000e-07, 5.00000e-07],
        [1.00000e+06, 1.00000e+06, 1.00000e+06, ..., 1.50125e-06,
         1.50125e-06, 1.50125e-06],
        [1.00000e+06, 1.00000e+06, 1.00000e+06, ..., 2.49875e-06,
         2.49875e-06, 2.49875e-06]]) * ksecond,
 'sum_inputs': array([[0.     , 0.     , 0.     , ..., 0.5    , 0.5    , 0.5    ],
        [0.     , 0.     , 0.     , ..., 0.50125, 0.50125, 0.50125],
        [0.     , 0.     , 0.     , ..., 0.49875, 0.49875, 0.49875]]),
 'spikes_received': array([[0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.]]),
 'global_clocks': array([[1.000e-03, 2.000e-03, 3.000e-03, ..., 4.998e+00, 4.999e+00,
         5.000e+00],
        [1.000e-03, 2.000e-03, 3.000e-03, ..., 4.998e+00, 4.999e+00,
         5.000e+

In [52]:
def compute_loss_and_gradients(results, target_output, w1, w2):
    """
    Compute gradients for w1 and w2 using spike timing surrogate gradient approximation.
    
    results: dict returned by mini1x1x1()
    target_output: target scalar value
    w1, w2: current weights
    
    Returns: loss, grad_w1, grad_w2
    """
    EPS = 1e-9
    
    sum_inputs = results['sum_inputs']
    spikes_received = results['spikes_received']
    
    # Neuron indices: 0=input, 1=hidden, 2=output
    # Use last recorded values (final time step)
    
    sum_output = sum_inputs[2, -1]
    recv_output = spikes_received[2, -1] + EPS
    scheduled_output_time = (sum_output / recv_output) + 2.0  # + layer (2)
    
    # Simple prediction mapping (inverse timing)
    pred_output = 1.0 - ((scheduled_output_time - 2.0) / 1.0)
    
    # Compute loss (MSE)
    loss = (pred_output - target_output) ** 2
    
    # Gradient of loss wrt prediction
    dL_dpred = 2 * (pred_output - target_output)
    
    # Gradient of prediction wrt scheduled_output_time is -1
    d_pred_dtime = -1.0
    
    # Calculate dt/dw2 (output synapse weight)
    # Use global_clock = layer index (2), as a proxy
    dt_dw2 = d_spike_timing_dw(w2, 2.0)
    
    grad_w2 = dL_dpred * d_pred_dtime * dt_dw2
    
    # For w1 (input->hidden), we chain through output
    dt_dw1 = d_spike_timing_dw(w1, 1.0)  # hidden layer global_clock=1.0
    
    # Backpropagate through output weight gradient
    grad_w1 = dL_dpred * d_pred_dtime * dt_dw2 * dt_dw1
    
    return loss, grad_w1, grad_w2


# Training loop:
inputs = np.array([0.9])
target = 0.2
w1 = 2.0
w2 = 5.0
lr = 0.1

for epoch in range(20):
    results = mini1x1x1(inputs, w1, w2)
    print(results)
    
    loss, grad_w1, grad_w2 = compute_loss_and_gradients(results, target, w1, w2)
    
    w1 -= lr * grad_w1
    w2 -= lr * grad_w2
    
    print(f"Epoch {epoch}: Loss = {loss:.6f}, w1 = {w1:.6f}, w2 = {w2:.6f}")
    #print("output:", results['spike_trains'][1])  # Output neuron spikes


{'spike_trains': {0: array([], dtype=float64) * second, 1: array([], dtype=float64) * second, 2: array([], dtype=float64) * second}, 'scheduled_times': array([[1.e+03, 1.e+03, 1.e+03, ..., 5.e-10, 5.e-10, 5.e-10],
       [1.e+03, 1.e+03, 1.e+03, ..., 1.e+03, 1.e+03, 1.e+03],
       [1.e+03, 1.e+03, 1.e+03, ..., 1.e+03, 1.e+03, 1.e+03]]) * Msecond, 'sum_inputs': array([[0. , 0. , 0. , ..., 0.5, 0.5, 0.5],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]]), 'spikes_received': array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'input_times': array([0.9]), 'mon_times': array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.997e+03, 4.998e+03,
       4.999e+03]) * usecond}
Epoch 0: Loss = 0.640000, w1 = 2.000000, w2 = 4.999999
output: [] s
{'spike_trains': {0: array([], dtype=float64) * second, 1: array([], dtype=float64) * second, 2: array([], dtype=float64) * second}, 'scheduled_times': array([[

In [ ]:
# from brian2 import *
# import numpy as np
# import logging, warnings

# prefs.codegen.target = 'cython'
# set_device('runtime')
# warnings.filterwarnings('ignore', category=RuntimeWarning)
# np.seterr(over='ignore', under='ignore')
# logging.getLogger('brian2').setLevel(logging.ERROR)

# start_scope()

# defaultclock.dt = 0.001*ms  

# # Custom timing function
# @implementation('numpy', discard_units=True)
# @check_units(w=1, global_clock=1, result=1)
# def spike_timing(w, global_clock):
#     x = (global_clock % 1)
#     z = 5.0 * (x - 0.5)  # Smoother sigmoid
#     sigmoid_val = 1.0 / (1.0 + np.exp(-w * z))
#     return sigmoid_val

# @implementation('numpy', discard_units=True)
# @check_units(w=1, global_clock=1, result=1)
# def d_spike_timing_dw(w, global_clock):
#     x = (global_clock % 1)
#     z = 5.0 * (x - 0.5)
#     sig = 1.0 / (1.0 + np.exp(-w * z))
#     return sig * (1.0 - sig) * z

# def mini1x1x1(inputs, w1, w2):
#     """
#     Mini 1x1x1 network with a single input, hidden, and output neuron.
#     Returns additional information needed for proper backpropagation.
#     """
#     # Define network sizes
#     n_input = 1
#     n_hidden = 1
#     n_output = 1
#     n_total = n_input + n_hidden + n_output

#     # Full neuron group
#     neurons = NeuronGroup(n_total, '''
#         v : 1
#         sum : 1
#         spikes_received : 1
#         scheduled_time : second
#         global_clock : 1
#         spiked : boolean
#     ''', threshold='v > 1', reset='''
#     v = 0
#     spiked = True
#     ''', method='exact')

#     neurons.v = 0
#     neurons.scheduled_time = 1e9 * second
#     neurons.global_clock = 0.0
#     neurons.sum = 0.0
#     neurons.spikes_received = 0.0

#     # Spike inputs     
#     stim = SpikeGeneratorGroup(n_input, indices=range(n_input), times=(inputs) * ms)

#     # Input → Hidden connections
#     syn_input = Synapses(stim, neurons[0:n_input], '''
#         w : 1
#         layer : 1
#     ''', on_pre='''
#         spikes_received += 1
#         sum += spike_timing(w, global_clock)
#         scheduled_time = ((sum/spikes_received) + layer) * ms 
#     ''')
#     syn_input.connect(j='i')
#     syn_input.layer = 0

#     # Hidden layer: input → hidden
#     syn_hidden = Synapses(neurons[0:n_input], neurons[n_input:n_input + n_hidden], '''
#         w : 1
#         layer : 1
#     ''', on_pre='''
#         spikes_received += 1
#         sum += spike_timing(w, global_clock)
#         scheduled_time = ((sum/spikes_received) + layer) * ms 
#     ''')
#     syn_hidden.connect()
#     syn_hidden.w = w1 
#     syn_hidden.layer = 1

#     # Output layer: hidden → output
#     syn_output = Synapses(neurons[n_input:n_input + n_hidden], neurons[n_input + n_hidden:n_total], '''
#         w : 1
#         layer : 1
#     ''', on_pre='''
#         spikes_received += 1
#         sum += spike_timing(w, global_clock)
#         scheduled_time = ((sum/spikes_received) + layer) * ms 
#     ''')
#     syn_output.connect()
#     syn_output.w = w2 
#     syn_output.layer = 2

#     neurons.run_regularly('''
#         v = (1.0 - spiked) * int(abs(t - scheduled_time) < 0.005*ms) * 1.2
#         global_clock += 0.001
#     ''', dt=0.001*ms)

#     # Monitors
#     mon = StateMonitor(neurons, 'v', record=True, dt=0.001*ms)
#     mon_sum = StateMonitor(neurons, 'sum', record=True)
#     sp_mon = StateMonitor(neurons, 'spikes_received', record=True)
#     sch_time = StateMonitor(neurons, 'scheduled_time', record=True)
#     clock_mon = StateMonitor(neurons, 'global_clock', record=True)

#     spikemon = SpikeMonitor(neurons)

#     run(5*ms)

#     # Debug output
#     for i in range(n_total):
#         times = spikemon.spike_trains()[i]
#         if len(times) > 0:
#             formatted_times = [f"{t/ms:.3f} ms" for t in times]
#             print(f"Neuron {i} spike times: {formatted_times}")
            
#     return {
#         'spike_trains': spikemon.spike_trains(),
#         'scheduled_times': sch_time.scheduled_time[:],
#         'sum_inputs': mon_sum.sum[:],
#         'spikes_received': sp_mon.spikes_received[:],
#         'global_clocks': clock_mon.global_clock[:],
#         'input_times': inputs,
#         'mon_times': mon.t[:],
#         'weights': {'w1': w1, 'w2': w2}
#     }

# def compute_loss_and_gradients(results, target_output, w1, w2):
#     """
#     Simplified and corrected gradient computation.
#     """
#     EPS = 1e-9
    
#     sum_inputs = results['sum_inputs']
#     spikes_received = results['spikes_received']
    
#     # Get final state values for output neuron (index 2)
#     sum_output = sum_inputs[2, -1]
#     recv_output = max(spikes_received[2, -1], 1.0)  # Ensure at least 1 to avoid division issues
    
#     # Calculate average contribution per spike
#     avg_contribution = sum_output / recv_output
    
#     # Simple prediction mapping - use the average contribution directly
#     # Scale it to [0,1] range assuming contributions are typically [0,1]
#     pred_output = np.clip(avg_contribution, 0.0, 1.0)
    
#     # Loss computation
#     loss = (pred_output - target_output) ** 2
    
#     # Gradient of loss w.r.t. prediction
#     dL_dpred = 2 * (pred_output - target_output)
    
#     # Debug: print intermediate values
#     print(f"    Debug: sum_output={sum_output:.6f}, recv_output={recv_output:.6f}, avg_contrib={avg_contribution:.6f}")
    
#     # For w2: how does changing w2 affect the sum at output?
#     # The sum comes from spike_timing(w2, global_clock) calls
#     # Use the actual global clock value when the output layer is active
#     output_clock_time = 2.0  # Layer 2 gets global_clock around 2.0
    
#     # Direct gradient of spike_timing w.r.t. w2
#     dsum_dw2 = d_spike_timing_dw(w2, output_clock_time)
    
#     # Gradient w.r.t. w2 (direct path)
#     grad_w2 = dL_dpred * (1.0 / recv_output) * dsum_dw2
    
#     # For w1: changing w1 affects the hidden neuron's timing, which affects when
#     # it spikes, which affects the output neuron's sum
#     # This is more complex - w1 affects the hidden neuron's scheduled time,
#     # which changes when it spikes, which changes the global_clock value
#     # seen by the output layer
    
#     hidden_clock_time = 1.0  # Layer 1 gets global_clock around 1.0
#     dsum_dw1 = d_spike_timing_dw(w1, hidden_clock_time)
    
#     # The effect of w1 on output is indirect through the hidden neuron
#     # Approximate this as: change in hidden timing affects output timing
#     grad_w1 = dL_dpred * (1.0 / recv_output) * dsum_dw1 * 0.5  # Dampening factor
    
#     # Much more conservative clipping
#     grad_w1 = np.clip(grad_w1, -1.0, 1.0)
#     grad_w2 = np.clip(grad_w2, -1.0, 1.0)
    
#     return loss, grad_w1, grad_w2

# # Training loop with improved parameters
# def train_network():
#     inputs = np.array([0.9])
#     target = 0.2
#     w1 = 0.5
#     w2 = 0.5
#     lr = 0.001  # Much smaller learning rate
    
#     print("Starting training...")
#     print(f"Initial: w1={w1:.6f}, w2={w2:.6f}")
    
#     for epoch in range(20):
#         results = mini1x1x1(inputs, w1, w2)
#         loss, grad_w1, grad_w2 = compute_loss_and_gradients(results, target, w1, w2)
        
#         # Update weights
#         w1 -= lr * grad_w1
#         w2 -= lr * grad_w2
        
#         # Get current prediction for monitoring
#         sum_output = results['sum_inputs'][2, -1]
#         recv_output = max(results['spikes_received'][2, -1], 1.0)
#         pred_output = np.clip(sum_output / recv_output, 0.0, 1.0)
        
#         print(f"Epoch {epoch:2d}: Loss={loss:.6f}, Pred={pred_output:.6f}, Target={target:.6f}")
#         print(f"         w1={w1:.6f}, w2={w2:.6f}, grad_w1={grad_w1:.6f}, grad_w2={grad_w2:.6f}")
        
#         # Early stopping if loss is very small
#         if loss < 1e-6:
#             print("Converged!")
#             break
    
#     return w1, w2

# # Run the training
# if __name__ == "__main__":
#     final_w1, final_w2 = train_network()
#     print(f"\nFinal weights: w1={final_w1:.6f}, w2={final_w2:.6f}")

Starting training...
Initial: w1=0.500000, w2=0.500000
    Debug: sum_output=0.000000, recv_output=1.000000, avg_contrib=0.000000
Epoch  0: Loss=0.040000, Pred=0.000000, Target=0.200000
         w1=0.499913, w2=0.499827, grad_w1=0.086552, grad_w2=0.173105
    Debug: sum_output=0.000000, recv_output=1.000000, avg_contrib=0.000000
Epoch  1: Loss=0.040000, Pred=0.000000, Target=0.200000
         w1=0.499827, w2=0.499654, grad_w1=0.086563, grad_w2=0.173146
    Debug: sum_output=0.000000, recv_output=1.000000, avg_contrib=0.000000
Epoch  2: Loss=0.040000, Pred=0.000000, Target=0.200000
         w1=0.499740, w2=0.499481, grad_w1=0.086573, grad_w2=0.173188
    Debug: sum_output=0.000000, recv_output=1.000000, avg_contrib=0.000000
Epoch  3: Loss=0.040000, Pred=0.000000, Target=0.200000
         w1=0.499654, w2=0.499307, grad_w1=0.086584, grad_w2=0.173229
    Debug: sum_output=0.000000, recv_output=1.000000, avg_contrib=0.000000
Epoch  4: Loss=0.040000, Pred=0.000000, Target=0.200000
         w

In [7]:
# @implementation('numpy', discard_units=True)
# @check_units(w=1, global_clock=1, result=1)
# def spike_timing(w, global_clock):
#     x = (global_clock % 1)
#     z = 5.0 * (x - 0.5)  # Smoother sigmoid
#     sigmoid_val = 1.0 / (1.0 + np.exp(-w * z))
#     return sigmoid_val

# @implementation('numpy', discard_units=True)
# @check_units(w=1, global_clock=1, result=1)
# def d_spike_timing_dw(w, global_clock):
#     x = (global_clock % 1)
#     z = 5.0 * (x - 0.5)
#     sig = 1.0 / (1.0 + np.exp(-w * z))
#     return sig * (1.0 - sig) * z


spike_timing(10.0, 0.2)


weight=  10.0
x=  0.2


3.059022269256247e-07